# Automated Address Lookup and Geocoding



In [11]:
#! pip install --upgrade google-maps-places
#! pip install --upgrade geopandas
#! pip install --upgrade fiona

In [3]:
from google.maps import places_v1
import pandas as pd

In [4]:
client = places_v1.PlacesClient()
organizations = ["Circa 1877", "Jane Bond", "Princess Cafe", "AOK Arcade Bar", "Full Circle Foods"]

In [7]:
addresses = []
latitudes = []
longitudes = []
types = []
display_names = []
primary_types = []
phone_numbers = []
for organization in organizations:
    request = places_v1.SearchTextRequest(
        text_query = organization + "Region of Waterloo",
    )
    response = client.search_text(request=request, 
                                  metadata=[("x-goog-fieldmask", 
                                             "places.displayName,places.formattedAddress,places.location,places.types,places.displayName,places.primaryType,places.nationalPhoneNumber")])
    addresses.append(response.places[0].formatted_address)
    longitudes.append(response.places[0].location.longitude)
    latitudes.append(response.places[0].location.latitude)
    types.append(response.places[0].types)
    display_names.append(response.places[0].display_name)
    primary_types.append(response.places[0].primary_type)
    phone_numbers.append(response.places[0].national_phone_number)
location_data = pd.DataFrame.from_dict({'org' : organizations,
                                        'address' : addresses,
                                        'longitude' : longitudes,
                                        'latitude' : latitudes,
                                        'types' : types,
                                        'formatted_name' : display_names,
                                        'primary_type' : primary_types,
                                        'phone' : phone_numbers
                                       })
location_data

,org,address,longitude,latitude,types,formatted_name,primary_type,phone
0,Circa 1877,"181 King St S, Waterloo, ON N2J 0E7, Canada",-80.520197,43.460381,"[point_of_interest, establishment]","text: ""Circa1877""\nlanguage_code: ""en""\n",,
1,Jane Bond,"5 Princess St W, Waterloo, ON N2L 2X7, Canada",-80.523333,43.466634,"[vegetarian_restaurant, bar, restaurant, food,...","text: ""Jane Bond""\nlanguage_code: ""en""\n",vegetarian_restaurant,(519) 886-1689
2,Princess Cafe,"46 King St N, Waterloo, ON N2J 2W8, Canada",-80.522579,43.466499,"[cafe, coffee_shop, store, restaurant, food, p...","text: ""Princess Cafe""\nlanguage_code: ""en""\n",cafe,(519) 886-0227
3,AOK Arcade Bar,"165 King St W Unit B03, Kitchener, ON N2G 1A7,...",-80.492287,43.450546,"[bar, point_of_interest, establishment]","text: ""AOK Craft Beer + Arcade""\nlanguage_code...",bar,
4,Full Circle Foods,"16 King St E, Kitchener, ON N2G 2K6, Canada",-80.488748,43.449815,"[grocery_store, store, food, point_of_interest...","text: ""Full Circle Foods""\nlanguage_code: ""en""\n",grocery_store,(519) 744-5331


In [8]:
location_data.to_csv('output_data/craigs_favourite_places.csv', index = False)

# Convert to Shapefile


In [1]:
import geopandas as gpd
from shapely.geometry import Point

In [11]:
location_data['geometry'] = location_data.apply(lambda x: Point((float(x.longitude), float(x.latitude))), axis=1)
shapefile = gpd.GeoDataFrame(location_data[["org", "address", "phone", "geometry"]], geometry='geometry')
shapefile.to_file("./output_data/craigs_favourite_places.shp", 
                  driver='ESRI Shapefile', 
                  engine = "fiona", 
                  crs = "+proj=longlat +datum=WGS84")